In [0]:
from preprocessing import *
from clustering import *
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from GMM import my_GMM_uniform_cluster
data_contrats = pd.read_csv('train_contrats_approx.csv',sep=",")
from sklearn.metrics import adjusted_rand_score
from collections import Counter

Le but est ici d'évaluer la stabilité de differentes méthodes de clustering, en utilisant le procédé suivant : 
Clustering sur un jeu d'entrainement (75%) (Clustering A)
Nouveau clustering sur toute la base de données, ie 75% initaux + 25% restants (Clustering B)
Calcul du adjusted rand index, qui est d'autant plus grand que deux points dans un même cluster dan le clustering A sont toutjours dans le même cluster dans le clustering B, ou que deux points dans deux clusters différents dans le clustering A sont toutjours dans deux clusters différents dans le clustering B.
Lors du calcul du score, seuls 75% des points sont donc considérés.
Selon nous, une perturbation de 25% est forte, et permettera de vraiment mettre les méthodes de clustering à l'épreuve.

In [8]:
data = pd.read_csv("train_contrats_approx.csv", sep=",", engine="python")

# For original database ---
#data = pd.read_csv("train_contrats.csv", sep=";")
#data["Freq_sinistre"] = data["nombre_de_sinistre"]/data["Exposition_au_risque"]

#train_data, _ = preprocessing(data, balance=False, train_size=0.75)
preprocessing(data, balance=False, train_size=1.)[0]
data = preprocessing(data, balance=False, train_size=1.)[0]
train_data = data[:(int(len(data)*0.75))]

/content/preprocessing.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clustering["Classe_Age_Situ_Cont"]=data_clustering["Classe_Age_Situ_Cont"].apply(classe_age)
/content/preprocessing.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clustering["franchise"]=data_clustering["franchise"].apply(franchise_)
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [9]:
train_data

,Classe_Age_Situ_Cont,franchise,Exposition_au_risque,ValeurPuissance,Freq_sinistre,Type_Apporteur_1,Type_Apporteur_2,Type_Apporteur_3,Activite_1,Activite_2,Activite_3,Activite_4,Activite_5,Activite_6,Activite_7,Activite_8,Mode_gestion_N,Mode_gestion_P,Zone_1,Zone_2,Zone_3,Zone_4,Zone_5,Zone_6,Fractionnement_A,Fractionnement_S,Fractionnement_T,FORMULE_2,FORMULE_3,FORMULE_4,FORMULE_5,Age_du_vehicule_<= 5 ans,Age_du_vehicule_> 5 ans
0,0.596130,0.298065,0.054645,0.695485,0.000183,0.099355,0.000000,0.0,0.0,0.0,0.099355,0.0,0.0,0.0,0.000000,0.0,0.099355,0.000000,0.000000,0.0,0.0,0.099355,0.0,0.0,0.099355,0.000000,0.000000,0.0,0.000000,0.0,0.099355,0.099355,0.000000
1,0.541791,0.216716,0.044427,0.758507,0.000274,0.000000,0.108358,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.108358,0.0,0.108358,0.000000,0.000000,0.0,0.0,0.108358,0.0,0.0,0.000000,0.108358,0.000000,0.0,0.108358,0.0,0.000000,0.000000,0.108358
2,0.542157,0.216863,0.024939,0.759020,0.000262,0.000000,0.108431,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.108431,0.0,0.108431,0.000000,0.000000,0.0,0.0,0.108431,0.0,0.0,0.000000,0.108431,0.000000,0.0,0.108431,0.0,0.000000,0.000000,0.108431
3,0.692925,0.277170,0.036032,0.554340,0.000773,0.000000,0.138585,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.138585,0.0,0.138585,0.000000,0.000000,0.0,0.0,0.138585,0.0,0.0,0.000000,0.138585,0.000000,0.0,0.000000,0.0,0.138585,0.138585,0.000000
4,0.755575,0.251858,0.030223,0.503717,0.004656,0.000000,0.125929,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.125929,0.0,0.125929,0.000000,0.000000,0.0,0.0,0.125929,0.0,0.0,0.000000,0.125929,0.000000,0.0,0.000000,0.0,0.125929,0.125929,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65416,0.110427,0.331280,0.110427,0.883414,0.009290,0.110427,0.000000,0.0,0.0,0.0,0.110427,0.0,0.0,0.0,0.000000,0.0,0.000000,0.110427,0.110427,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.110427,0.0,0.110427,0.0,0.000000,0.110427,0.000000
65417,0.110510,0.331530,0.103879,0.884080,0.000000,0.110510,0.000000,0.0,0.0,0.0,0.110510,0.0,0.0,0.0,0.000000,0.0,0.000000,0.110510,0.110510,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.110510,0.0,0.110510,0.0,0.000000,0.110510,0.000000
65418,0.216923,0.325384,0.108461,0.867690,0.008504,0.108461,0.000000,0.0,0.0,0.0,0.108461,0.0,0.0,0.0,0.000000,0.0,0.000000,0.108461,0.108461,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.108461,0.0,0.108461,0.0,0.000000,0.000000,0.108461
65419,0.239046,0.358569,0.119523,0.836660,0.000000,0.119523,0.000000,0.0,0.0,0.0,0.119523,0.0,0.0,0.0,0.000000,0.0,0.000000,0.119523,0.119523,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.119523,0.0,0.119523,0.0,0.000000,0.119523,0.000000


#### Adjusted rand index

Pour avoir une baseline et mieux intepreter l'ARI, il est calculé ici pour un clustering **aléatoire**, pour un nombre de point à peu près égal à celui de notre base de données. L'ARI devrait dans ce cas valoir 0.

In [0]:
ari = 0
for i in range(100):
  clust_A = np.random.randint(low=0, high=4, size=65000)
  clust_B = np.random.randint(low=0, high=4, size=65000)
  ari += adjusted_rand_score(clust_A,clust_B)/100

In [0]:
print(ari)

9.029652763787604e-07


### GMM clustering

Let's try with 5 clusters at first.

Train data

In [0]:
t= time.time()
estimator = GaussianMixture(n_components=5,
              covariance_type='full', max_iter=500, random_state=0)
GMM_labels_train=estimator.fit_predict(train_data.drop(["Freq_sinistre"],axis=1))
elapsed = time.time()-t
print("time elapsed : {} s".format(elapsed))

time elapsed : 6.481006860733032 s


In [0]:
Counter(GMM_labels_train)

Counter({0: 1108, 1: 41060, 2: 8147, 3: 6406, 4: 8700})

All data

In [0]:
t= time.time()
estimator = GaussianMixture(n_components=5,
              covariance_type='full', max_iter=500, random_state=0)
GMM_labels_all=estimator.fit_predict(data.drop(["Freq_sinistre"],axis=1))
elapsed = time.time()-t
print("time elapsed : {} s".format(elapsed))

time elapsed : 7.044588088989258 s


We select the point of train_data for comparison

In [0]:
GMM_labels_all = GMM_labels_all[:len(GMM_labels_train)]
adjusted_rand_score(GMM_labels_train,GMM_labels_all)

0.2477805078878556

Une ARI de 0.25 est basse. Le clustering par GMM semble peu fiable. Nous allons essayer de calculer cette valeur pour différent nombres de clusters.

In [0]:
for n_clusters in range(1,10):

  estimator_train = GaussianMixture(n_components=n_clusters,
              covariance_type='full', max_iter=500, random_state=0)
  GMM_labels_train=estimator_train.fit_predict(train_data.drop(["Freq_sinistre"],axis=1))

  estimator = GaussianMixture(n_components=n_clusters,
              covariance_type='full', max_iter=500, random_state=0)
  GMM_labels_all=estimator.fit_predict(data.drop(["Freq_sinistre"],axis=1))

  GMM_labels_all = GMM_labels_all[:len(GMM_labels_train)]
  ari = adjusted_rand_score(GMM_labels_train,GMM_labels_all)
  print(f"{n_clusters} clusters : ARI = {ari}")

1 clusters : ARI = 1.0
2 clusters : ARI = 0.2181959753845813
3 clusters : ARI = 0.370497276938162
4 clusters : ARI = 0.46649178384124035
5 clusters : ARI = 0.2477805078878556
6 clusters : ARI = 0.11815108600122244
7 clusters : ARI = 0.21359781429012634
8 clusters : ARI = 0.5589418104416999
9 clusters : ARI = 0.29430320638946644


4 cluster maximise le ARI

#### Profil moyen de chaque cluster

In [0]:
estimator = GaussianMixture(n_components=4,
              covariance_type='full', max_iter=500, random_state=0)
GMM_labels_train=estimator.fit_predict(data.drop(["Freq_sinistre"],axis=1))

In [0]:
data["cluster"] = GMM_labels_train

In [20]:
data

,Classe_Age_Situ_Cont,franchise,Exposition_au_risque,ValeurPuissance,Freq_sinistre,Type_Apporteur_1,Type_Apporteur_2,Type_Apporteur_3,Activite_1,Activite_2,Activite_3,Activite_4,Activite_5,Activite_6,Activite_7,Activite_8,Mode_gestion_N,Mode_gestion_P,Zone_1,Zone_2,Zone_3,Zone_4,Zone_5,Zone_6,Fractionnement_A,Fractionnement_S,Fractionnement_T,FORMULE_2,FORMULE_3,FORMULE_4,FORMULE_5,Age_du_vehicule_<= 5 ans,Age_du_vehicule_> 5 ans,cluster,count
0,0.596130,0.298065,0.054645,0.695485,0.000183,0.099355,0.000000,0.0,0.0,0.0,0.099355,0.0,0.0,0.000000,0.000000,0.0,0.099355,0.0,0.0,0.000000,0.0,0.099355,0.0,0.0,0.099355,0.000000,0.000000,0.0,0.000000,0.0,0.099355,0.099355,0.000000,3,1
1,0.541791,0.216716,0.044427,0.758507,0.000274,0.000000,0.108358,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.108358,0.0,0.108358,0.0,0.0,0.000000,0.0,0.108358,0.0,0.0,0.000000,0.108358,0.000000,0.0,0.108358,0.0,0.000000,0.000000,0.108358,1,1
2,0.542157,0.216863,0.024939,0.759020,0.000262,0.000000,0.108431,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.108431,0.0,0.108431,0.0,0.0,0.000000,0.0,0.108431,0.0,0.0,0.000000,0.108431,0.000000,0.0,0.108431,0.0,0.000000,0.000000,0.108431,1,1
3,0.692925,0.277170,0.036032,0.554340,0.000773,0.000000,0.138585,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.138585,0.0,0.138585,0.0,0.0,0.000000,0.0,0.138585,0.0,0.0,0.000000,0.138585,0.000000,0.0,0.000000,0.0,0.138585,0.138585,0.000000,3,1
4,0.755575,0.251858,0.030223,0.503717,0.004656,0.000000,0.125929,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.125929,0.0,0.125929,0.0,0.0,0.000000,0.0,0.125929,0.0,0.0,0.000000,0.125929,0.000000,0.0,0.000000,0.0,0.125929,0.125929,0.000000,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87223,0.612122,0.204041,0.028566,0.714142,0.001815,0.102020,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.102020,0.000000,0.0,0.102020,0.0,0.0,0.102020,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.102020,0.0,0.102020,0.0,0.000000,0.102020,0.000000,2,1
87224,0.612122,0.204041,0.028566,0.714142,0.001815,0.102020,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.102020,0.000000,0.0,0.102020,0.0,0.0,0.102020,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.102020,0.0,0.102020,0.0,0.000000,0.102020,0.000000,2,1
87225,0.612291,0.204097,0.016328,0.714339,0.000694,0.102048,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.102048,0.000000,0.0,0.102048,0.0,0.0,0.102048,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.102048,0.0,0.102048,0.0,0.000000,0.000000,0.102048,2,1
87226,0.612122,0.204041,0.028566,0.714142,0.001815,0.102020,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.102020,0.000000,0.0,0.102020,0.0,0.0,0.102020,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.102020,0.0,0.102020,0.0,0.000000,0.102020,0.000000,2,1


In [22]:
print(Counter(GMM_labels_train))
data.groupby('cluster').mean()

Counter({2: 46009, 0: 24367, 1: 14753, 3: 2099})


,Classe_Age_Situ_Cont,franchise,Exposition_au_risque,ValeurPuissance,Freq_sinistre,Type_Apporteur_1,Type_Apporteur_2,Type_Apporteur_3,Activite_1,Activite_2,Activite_3,Activite_4,Activite_5,Activite_6,Activite_7,Activite_8,Mode_gestion_N,Mode_gestion_P,Zone_1,Zone_2,Zone_3,Zone_4,Zone_5,Zone_6,Fractionnement_A,Fractionnement_S,Fractionnement_T,FORMULE_2,FORMULE_3,FORMULE_4,FORMULE_5,Age_du_vehicule_<= 5 ans,Age_du_vehicule_> 5 ans,count
cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.430953,0.334800,0.057890,0.714585,0.001946,0.096762,0.025351,0.001472,0.095080,0.015792,0.000000,0.000000,0.012713,0.000000,0.000000,0.000000,0.069350,0.054235,0.027650,0.025772,0.044185,0.012389,0.013590,0.000000,0.054064,0.027531,0.041990,0.0000,0.091530,0.001472,0.030583,0.080462,0.043123,1
1,0.371280,0.373315,0.071697,0.724712,0.001785,0.046207,0.025437,0.055523,0.011566,0.000000,0.042277,0.009672,0.000000,0.014518,0.031654,0.017481,0.052847,0.074319,0.009117,0.010272,0.020289,0.042023,0.045466,0.000000,0.031288,0.039686,0.056193,0.0000,0.071644,0.055523,0.000000,0.092292,0.034874,1
2,0.416449,0.327935,0.048561,0.729190,0.001885,0.096601,0.026183,0.000000,0.000037,0.000004,0.030255,0.016383,0.000000,0.023826,0.033819,0.018460,0.094384,0.028399,0.038866,0.027246,0.056641,0.000008,0.000019,0.000002,0.036607,0.027371,0.058806,0.0001,0.110066,0.000000,0.012618,0.078676,0.044107,1
3,0.431697,0.357852,0.056714,0.705802,0.001364,0.062318,0.051606,0.007607,0.015905,0.003249,0.025437,0.015275,0.002195,0.010339,0.025828,0.023303,0.074306,0.047224,0.001597,0.002225,0.002369,0.027959,0.034330,0.053051,0.033114,0.040453,0.047963,0.0000,0.041188,0.010013,0.070330,0.088532,0.032999,1
